In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

### Agents

In [2]:
topic = input("Enter a topic: ")

In [3]:
dave_agent = Agent(
    name="Dave",
    instructions="You are Dave Chappelle. You have watched every Dave Chappelle standup and you just can't help yourself to make everything feel like it is from the mind of Dave Chappelle.",
    model="gpt-4o-mini"
)

result_dave = await Runner.run(dave_agent, topic)


NameError: name 'Agent' is not defined

In [ ]:
kevin_agent = Agent(
    name="Kevin",
    instructions="You are Kevin Hart. You have watched every Kevin Hart standup and you just can't help yourself to make everything feel like it is from the mind of Kevin Hart.",
    model="gpt-4o-mini"
)

result_kevin = await Runner.run(kevin_agent, topic)


In [ ]:
judge_agent = Agent(
      name="Judge",
      instructions="""You are a comedy judge. You evaluate jokes based on:
  - Timing and delivery (as conveyed through text)
  - Originality
  - How well it captures the comedian's voice
  - Actually being funny

  Be honest. Pick a winner. Explain why.""",
      model="gpt-4o-mini"
  )

prompt = f"""Judge these two comedy takes on {topic} and return result in well formatted markdown:

  DAVE CHAPPELLE:
  {result_dave.final_output}

  KEVIN HART:
  {result_kevin.final_output}

  Who did it better?"""

result_judge = await Runner.run(judge_agent, prompt)
print("Dave's joke:")
print(result_dave.final_output)
print("Kevin's joke:")
print(result_kevin.final_output)
print("Judge's verdict:")
print(result_judge.final_output)

Dave's joke:
Man, kids can be the real MVPs of mischief, can’t they? It’s like they come out the womb with a map of all the trouble spots in your house! You ever notice how they have that innate ability to turn a simple living room into a disaster zone? One minute, you’re chillin’ on the couch, and the next, you’re dodging Legos like you're in a ninja training camp.

And they ask the craziest questions! "Why is the sky blue?" “Why do we have to wear pants?” Like, I don’t know, kid! I was just trying to figure out why adults think they can charge $5 for a bottle of water!

But at the end of the day, they keep us humble. One minute you’re the king of your castle, and the next, they remind you that you’re just a glorified snack-fetching machine. Kids are the ultimate reality check—like a tiny, hilarious version of the IRS. You think you’re doing great until they come in with that “Why don’t we have a pet dinosaur?” And suddenly, you’re questioning your entire life choices.

So yeah, kids 

### Structured Output

In [ ]:
from pydantic import BaseModel
from agents import Agent, Runner

class Recipe(BaseModel):
    name: str
    ingredients: list[str]
    instructions: list[str]
    cooking_time: int
    servings: int

agent = Agent(
    name="Chef",
    instructions="You are an expert in making Indian Cuisine. You will be given a name of a food and\n\nyou will have to provide a recipe for it.\n\nYour recipe should include the following details:\n\n- A list of ingredients\n- A list of instructions\n- The cooking time in minutes\n- The number of servings",
    output_type=Recipe, 
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Aaloo Paratha")
print(result.final_output)


name='Aaloo Paratha' ingredients=['2 cups whole wheat flour (atta)', '2 medium-sized potatoes (boiled and mashed)', '1 teaspoon cumin seeds', '2 green chilies (finely chopped)', '1/2 teaspoon garam masala', '1/2 teaspoon red chili powder', 'Salt to taste', 'Fresh coriander leaves (chopped)', 'Oil or ghee for frying', 'Water (as needed)'] instructions=['In a mixing bowl, combine the whole wheat flour and a pinch of salt. Gradually add water and knead to make a soft dough. Cover and let it rest for 20-30 minutes.', 'In another bowl, mix the boiled and mashed potatoes with cumin seeds, green chilies, garam masala, red chili powder, and fresh coriander leaves. Add salt to taste and mix well.', 'Divide the dough into equal-sized balls (about the size of a golf ball). Do the same with the potato filling mixture.', 'Take one dough ball, flatten it slightly, and place a potato ball in the center. Close the edges and seal tightly, forming a small pouch.', 'Dust the filled dough ball with some f

### Tool Calling

In [5]:
from agents import Agent, Runner, function_tool, WebSearchTool

@function_tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location."""
    print(f"Getting weather for {location}")
    return f"The current weather in {location} is sunny."

agent = Agent(
    name="WeatherBot",
    instructions="You are a helpful weather bot. You will be given a location and\n\nyou will have to provide the current weather for it and also based on the location roast the location for its hypocrisy in one line by finding recent news articles on the internet which should be date 2026",
    tools=[get_current_weather, WebSearchTool()],
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "What is the weather like in New Delhi")
print(result.final_output)



Getting weather for New Delhi
As of January 7, 2026, New Delhi is experiencing hazy sunshine with a temperature of 59°F (15°C).

## Weather for New Delhi, India:
Current Conditions: Hazy sunshine, 59°F (15°C)

Daily Forecast:
* Wednesday, January 7: Low: 46°F (8°C), High: 61°F (16°C), Description: Hazy and cool; air quality will be very unhealthy
* Thursday, January 8: Low: 44°F (6°C), High: 59°F (15°C), Description: Hazy and cool; air quality will be very unhealthy
* Friday, January 9: Low: 44°F (7°C), High: 62°F (17°C), Description: After a cloudy start, sunshine returns; air quality will be very unhealthy
* Saturday, January 10: Low: 44°F (7°C), High: 66°F (19°C), Description: Hazy; air quality will be very unhealthy
* Sunday, January 11: Low: 44°F (6°C), High: 63°F (17°C), Description: Hazy; air quality will be very unhealthy
* Monday, January 12: Low: 45°F (7°C), High: 66°F (19°C), Description: Mostly sunny; air quality will be very unhealthy
* Tuesday, January 13: Low: 44°F (7°C)

### Handoff

In [ ]:
from agents import Agent, Runner, function_tool, handoff, RunContextWrapper
from pydantic import BaseModel

class plan_execute(BaseModel):
    plan: str
    execute: str

executor_agent = Agent(
    name="ExecutorAgent",
    instructions=(
    "You are a helpful assistant that executes a task."
    ),
    model="gpt-4o-mini",
    output_type=plan_execute
)

plan_agent = Agent(
    name="PlanAgent",
    instructions=(
    "You are a helpful assistant that plans out a task."
    "After planning the task give it to the executor agent."
    ),
    model="gpt-4o-mini",
handoffs=[executor_agent]
)

executor_response = await Runner.run(plan_agent, "What is Langgraph")

print(executor_response.final_output)


Langgraph is a tool or platform designed for visualizing and working with language data, often used in natural language processing (NLP) tasks. It can help users understand the relationships between different linguistic elements or models, similar to how a graph framework would organize data.

Key features may include:

- **Visualization**: Displaying language structures or model relationships in a graphical format.
- **Integration**: Connecting with other NLP tools or libraries to enhance functionality.
- **Data Handling**: Facilitating the import, export, and manipulation of language datasets.
- **Interactive Elements**: Allowing users to interact with the data for better analysis and understanding.

If you need specific information or aspects about Langgraph, let me know!
